# Milestone Project 2 - Blackjack Game
In this milestone project you will be creating a Complete BlackJack Card Game in Python.

Here are the requirements:

* You need to create a simple text-based [BlackJack](https://en.wikipedia.org/wiki/Blackjack) game
* The game needs to have one player versus an automated dealer.
* The player can stand or hit.
* The player must be able to pick their betting amount.
* You need to keep track of the player's total money.
* You need to alert the player of wins, losses, or busts, etc...

And most importantly:

* **You must use OOP and classes in some portion of your game. You can not just use functions in your game. Use classes to help you define the Deck and the Player's hand. There are many right ways to do this, so explore it well!**


Feel free to expand this game. Try including multiple players. Try adding in Double-Down and card splits! Remember to you are free to use any resources you want and as always:

# HAVE FUN!

In [153]:
# https://tekeye.uk/playing_cards/svg-playing-cards

from IPython.display import display, HTML, SVG
from ipywidgets import Button, Layout

card_height = 150
card_width = card_height / 1.54385965
# card_a = SVG(url = 'https://tekeye.uk/playing_cards/images/svg_playing_cards/fronts/clubs_ace.svg')
card_a = Image(url = 'https://tekeye.uk/playing_cards/images/svg_playing_cards/fronts/png_96_dpi/clubs_ace.png', width=150)
no_wrap_div = '<div style="display: grid; gap: 8px; grid-template-columns: repeat(6, 150px);">{}{}{}{}{}{}{}{}</div>'
display(HTML(no_wrap_div.format(card_a.data, card_a.data, card_a.data, card_a.data, card_a.data, card_a.data, card_a.data, card_a.data)))

display(card_a)

card_deck = {
    "clubs_2": 2,
    "clubs_3": 3,
    "clubs_4": 4,
    "clubs_5": 5,
    "clubs_6": 6,
    "clubs_7": 7,
    "clubs_8": 8,
    "clubs_9": 9,
    "clubs_10": 10,
    "clubs_ace": 11,
    "clubs_jack": 10,
    "clubs_king": 10,
    "clubs_queen": 10,
    "diamonds_2": 2,
    "diamonds_3": 3,
    "diamonds_4": 4,
    "diamonds_5": 5,
    "diamonds_6": 6,
    "diamonds_7": 7,
    "diamonds_8": 8,
    "diamonds_9": 9,
    "diamonds_10": 10,
    "diamonds_ace": 11,
    "diamonds_jack": 10,
    "diamonds_king": 10,
    "diamonds_queen": 10,
    "hearts_2": 2,
    "hearts_3": 3,
    "hearts_4": 4,
    "hearts_5": 5,
    "hearts_6": 6,
    "hearts_7": 7,
    "hearts_8": 8,
    "hearts_9": 9,
    "hearts_10": 10,
    "hearts_ace": 11,
    "hearts_jack": 10,
    "hearts_king": 10,
    "hearts_queen": 10,
    "spades_2": 2,
    "spades_3": 3,
    "spades_4": 4,
    "spades_5": 5,
    "spades_6": 6,
    "spades_7": 7,
    "spades_8": 8,
    "spades_9": 9,
    "spades_10": 10,
    "spades_ace": 11,
    "spades_jack": 10,
    "spades_king": 10,
    "spades_queen": 10
}

print(card_a.data)

None
